In [1]:
import pandas as pd
import requests
from io import StringIO

# Importing the bird_master_df

In [9]:
%%time

bird_master_df = pd.read_csv('../data_sourcing/bird_master_df.csv')

bird_master_df = bird_master_df.drop(columns='Unnamed: 0')

bird_master_df

CPU times: user 15 s, sys: 2.09 s, total: 17.1 s
Wall time: 17.6 s


,image_url,image_id,scientific_name,species_no
0,http://www.taenos.com/img/ITIS/Dacnis-lineata-...,1,dacnis lineata,7319
1,http://41.media.tumblr.com/tumblr_m81kchMiQn1r...,8100,dacnis lineata,7319
2,http://madarbarat-lexikon.hu/images/dacnis%20c...,17844,dacnis lineata,7319
3,http://t3.gstatic.com/images?q=tbn:ANd9GcSG87H...,20557,dacnis lineata,7319
4,https://c2.staticflickr.com/4/3836/14737184263...,22123,dacnis lineata,7319
...,...,...,...,...
10357814,http://c5.statcounter.com/counter.php?sc_proje...,3371585,scolopax rosenbergi,181
10357815,https://c.statcounter.com/10601132/0/b39882cb/0/,3371585,scolopax rosenbergi,181
10357816,http://i74.servimg.com/u/f74/15/84/98/82/final...,3767414,scolopax rosenbergi,181
10357817,http://ecx.images-amazon.com/images/I/513ReDIt...,4612387,scolopax rosenbergi,181


# Importing the MVP #1 selection

In [11]:
# Importing list of selected birds provided by the DATA ANALYTICS team

fp_select = '../data_sourcing/bird_selection.csv'

selected_birds_df = pd.read_csv(fp_select, delimiter=',')

columns_tbd = ['']

# Rename the second column to 'scientific_name'
selected_birds_df = selected_birds_df.rename(columns={'scientific name': 'scientific_name','id': 'species_no'})

selected_birds_df = selected_birds_df[['species_no', 'scientific_name']]

# Display the modified DataFrame
selected_birds_df

,species_no,scientific_name
0,10705,geococcyx californianus
1,10923,cardinalis cardinalis
2,10381,fregata magnificens
3,4652,icterus pyrrhopterus
4,9402,buteo plagiatus
5,6142,chlamydotis undulata fuertaventurae
6,1084,prioniturus waterstradti
7,4067,myiobius sulphureipygius
8,3446,icterus melanopsis
9,10873,trichoglossus moluccanus


In [ ]:
# selected_birds_df.to_csv('selected_birds_df.csv', index=True)

In [12]:
selected_species_df = selected_birds_df.loc[0]
species = pd.DataFrame(selected_species_df).T
species

,species_no,scientific_name
0,10705,geococcyx californianus


In [13]:
check_df = bird_master_df[bird_master_df['species_no'] == species['species_no'].iloc[0]]
check_df

,image_url,image_id,scientific_name,species_no
9944672,http://thumb101.shutterstock.com/display_pic_w...,42258,geococcyx californianus,10705
9944673,1165818422,42258,geococcyx californianus,10705
9944674,3/stock-photo-greater-roadrunner-geococcyx-cal...,42258,geococcyx californianus,10705
9944675,http://www.basinandrangewatch.org/images/Visit...,50208,geococcyx californianus,10705
9944676,http://askabiologist.asu.edu/aab_tools/_media/...,61754,geococcyx californianus,10705
...,...,...,...,...
9945965,http://t2.gstatic.com/images?q=tbn:ANd9GcRyTdV...,4852653,geococcyx californianus,10705
9945966,http://s3.amazonaws.com/birdfellow-production/...,4855995,geococcyx californianus,10705
9945967,http://s3.amazonaws.com/birdfellow-production/...,4855995,geococcyx californianus,10705
9945968,http://4.bp.blogspot.com/_WK_jlizEEBQ/Sd_iK1f_...,4862856,geococcyx californianus,10705


# Data retrieval function

In [15]:
#### Current best shot!

import os
import requests
from PIL import Image
from io import BytesIO

def image_retrieval(bird_master_df, selected_birds_df, mydir=None, size = (256, 256), number = 1):
    
    # Temporary merged data frame based on a filtered version of bird_master_df
    temp_df = bird_master_df[bird_master_df['species_no'] == selected_birds_df['species_no'].iloc[0]]

    temp_list = []
    
    # Iterate over rows in temp_df
    for index, row in temp_df.iterrows():

        if row['image_id'] in temp_list:
            continue
        
        # Locate download links
        url_im = row['image_url']

        # Attempt download via link ('image_url') from temp_df
        try:
            response = requests.get(url_im)

        except:
            pass

        # Check if the download was successful
        if response.status_code == 200:
            
            try:
                im = Image.open(BytesIO(response.content))
            
                # Resize image using im.thumbnail(size, Image.ANTIALIAS) with 256 as the maximum dimension while keeping original image proportions
                im.thumbnail(size, Image.LANCZOS)
                
                # Create folder if it doesn't exist
                if mydir is None:
                    mydir = os.getcwd()
                    species_folder = os.path.join(mydir, f"{row['species_no']}")
                    os.makedirs(species_folder, exist_ok=True)
    
                # Save resulting image via im.save as [species_no]_[xxxx].png in the corresponding folder created above
                im_dir = os.path.join(species_folder, f"{row['species_no']}_{number:04d}.png")
                
                # Check if the file already exists, if not, save the image
                if not os.path.exists(im_dir):
                    im.save(im_dir)

                    ## remove duplicate image_ids from temp_df

                    
                    temp_list.append(row['image_id'])
                    
                    # 1. retrieve image_id of successfully saved image
                    #temp_img_id = row['image_id']
                    
                    # 2. Update temp_df and delete all rows below with matching image_id
                    # selec_birds_df.drop(0, inplace=True)
                    #temp_df = temp_df[temp_df['image_id'] != temp_img_id]
                    
                    print(f"Saved: {im_dir}")
                else:
                    print(f"Skipped (already exists): {im_dir}")
                
                # Increment the numbering variable for the next image
                number += 1
            
            except:
                
                pass

In [16]:
image_retrieval(check_df, species)

Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0001.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0002.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0003.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0004.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0005.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0006.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0007.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0008.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0009.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0010.png
Saved: /Users/tristanspilker/code/brunothormaehlen/birdies/notebooks/10705/10705_0011.png
Saved: /Us